<a href="https://www.kaggle.com/code/stutiiikatariaaa/kneighborsregressor?scriptVersionId=270015209" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
df = pd.read_csv('/kaggle/input/solargeneration/dataset.csv')
df['date_time'] = pd.to_datetime(df['date_time'], format='%d-%m-%Y %H:%M')

In [ ]:
# Feature engineering
df['hour'] = df['date_time'].dt.hour
df['day_of_week'] = df['date_time'].dt.dayofweek

# Select features and target
features = ['temperature', 'Ground_radiation_intensity ', 'hour']
X = df[features]
y = df['Photovoltaic_power_generation']
# Split data (time-series split)
train_size = int(len(X) * 0.8)
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from sklearn.model_selection import cross_val_score, KFold

# Initialize scaler
scaler = StandardScaler()

# Fit on training data only (avoid data leakage)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)  # Use same scaler

# Convert back to DataFrame for readability
X_train = pd.DataFrame(X_train_scaled, columns=features, index=X_train.index)
X_test = pd.DataFrame(X_test_scaled, columns=features, index=X_test.index)

K Neighbors Regressor without Hyperparameter Tuning

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

# Initialize KNN regressor with fixed parameters
knn = KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='minkowski')  # You can change these if needed

# Fit on training data
knn.fit(X_train, y_train)

# Predict on test set
y_pred_knn = knn.predict(X_test)

# Evaluate predictions
mse = mean_squared_error(y_test, y_pred_knn)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred_knn)
r2 = r2_score(y_test, y_pred_knn)

print(f"\nDefault KNN Test Set Evaluation:")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"R² Score: {r2:.2f}")




In [ ]:
# Visualization
results_knn = pd.DataFrame({
    'Actual': y_test,
    'Predicted_KNN': y_pred_knn
}, index=y_test.index)

plt.figure(figsize=(18, 6))
results_knn.head(500).plot(style=['-', '--'], ax=plt.gca())
plt.title("Default KNN: Actual vs. Predicted (First 500 Samples)", fontsize=16)
plt.xlabel("Time Index", fontsize=12)
plt.ylabel("Power Generation (kW)", fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

K Neighbors Regressor with Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid for KNN tuning
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

# Initialize KNN regressor
knn = KNeighborsRegressor()

# Setup GridSearchCV for KNN
grid_search_knn = GridSearchCV(knn, param_grid_knn, cv=5, scoring='r2', n_jobs=-1, verbose=2)

# Fit on training data
grid_search_knn.fit(X_train, y_train)

# Best estimator and parameters
print("Best parameters for KNN:", grid_search_knn.best_params_)
best_knn = grid_search_knn.best_estimator_

In [ ]:
# Predict on test set using best model
y_pred_knn = best_knn.predict(X_test)

# Evaluate predictions
mse_knn = mean_squared_error(y_test, y_pred_knn)
rmse_knn = np.sqrt(mse_knn)
mae_knn = mean_absolute_error(y_test, y_pred_knn)
r2_knn = r2_score(y_test, y_pred_knn)

print(f"\nTuned KNN Test Set Evaluation:")
print(f"MSE: {mse_knn:.2f}")
print(f"RMSE: {rmse_knn:.2f}")
print(f"MAE: {mae_knn:.2f}")
print(f"R² Score: {r2_knn:.2f}")


In [ ]:
# Visualization
import matplotlib.pyplot as plt
results_knn = pd.DataFrame({
    'Actual': y_test,
    'Predicted_KNN': y_pred_knn
}, index=y_test.index)

plt.figure(figsize=(18, 6))
results_knn.head(500).plot(style=['-', '--'], ax=plt.gca())
plt.title("Tuned KNN Regressor: Actual vs. Predicted (First 500 Samples)", fontsize=16)
plt.xlabel("Time Index", fontsize=12)
plt.ylabel("Power Generation (kW)", fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()